In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from k12libs.utils.nb_easy import k12ai_run_project

In [3]:
algo = 'gradient_boosting'
dataset = 'house-prices'

In [6]:
project = k12ai_run_project(debug=True, tb_port=9002,
    framework='k12ml', task='regressor', network=algo, dataset=dataset)

Box(children=(Box(children=(VBox(children=(Tab(children=(VBox(children=(HBox(children=(Text(value='regressor',…